## Notebook controller to update the state of Simulation


In [100]:
import time
import socket
import pickle

import numpy as np
import jax.numpy as jnp
from flax import serialization

from MultiAgentsSim.two_d_simulation import SimpleSimulation
from MultiAgentsSim.utils.network import SERVER

In [101]:
PORT = 5050
ADDR = (SERVER, PORT)
DATA_SIZE = 40000
EVAL_TIME = 10

color_map = {"r": (1.0, 0.0, 0.0),
             "g": (0.0, 1.0, 0.0),
             "b": (0.0, 0.0, 1.0)}

In [102]:
# Start the server and intialize connection

def connect_client():
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(ADDR)
    print(f"Connected to {ADDR}")

    msg = client.recv(1024).decode()
    state_example = pickle.loads(client.recv(DATA_SIZE))
    state_bytes_size = len(serialization.to_bytes(state_example))
    response = "NOTEBOOK"
    client.send(response.encode())
    time.sleep(1)

    return client, state_example, state_bytes_size

In [104]:
def close_client(client):
    client.send("CLOSE_CONNECTION".encode())

def get_state(client, state_example, state_bytes_size):
    client.send("GET_STATE".encode())
    response = client.recv(state_bytes_size)
    return serialization.from_bytes(state_example, response)

def change_state_agent_color(state, idx, color):
    colors = np.array(state.colors)
    colors[idx] = color_map[color]
    state = state.replace(colors=colors) 
    return state

def set_color(client, agent_idx, color, state_example, state_bytes_size):
    client.send("SET_STATE".encode())
    current_state = serialization.from_bytes(state_example, client.recv(state_bytes_size))
    response_state = change_state_agent_color(current_state, agent_idx, color)
    client.send(serialization.to_bytes(response_state))
    return 

In [105]:
client, state_example, state_bytes_size = connect_client()

Connected to ('localhost', 5050)


In [106]:
state = get_state(client, state_example, state_bytes_size)

In [108]:
# Apply this function with whatever agent_idx or color
set_color(client, 2, 'b', state_example, state_bytes_size)

In [109]:
close_client(client)